# Text Features In CatBoost

In [57]:

train_df, test_df = rotten_tomatoes()

train_df.head(2)

,id,synopsis,rating_MPAA,genre,director,writer,theater_date,dvd_date,box_office,runtime,studio,dvd_date_int,theater_date_int,review,rating,fresh,critic,top_critic,publisher,date,date_int,rating_10
0,830.0,A gay New Yorker stages a marriage of convenie...,R,Art House and International | Comedy | Drama |...,Ang Lee,Ang Lee | James Schamus | Neil Peng,1993-08-04,2004-06-15,NaN,111.0,NaN,20040615,19930804,NaN,0.800000,fresh,Carol Cling,0,Las Vegas Review-Journal,2004-04-16,20040416.0,8.0
1,1161.0,Screenwriter Nimrod Antal makes an impressive ...,R,Action and Adventure | Art House and Internati...,NaN,NaN,2005-04-01,2005-08-30,116783.0,105.0,ThinkFilm Inc.,20050830,20050401,"One very long, dark ride.",0.647059,rotten,NaN,0,E! Online,2005-04-22,20050422.0,6.0


In [0]:
auxiliary_columns = ['id', 'theater_date', 'dvd_date', 'rating', 'date']
cat_features = ['rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic', 'publisher']
text_features = ['synopsis', 'genre', 'director', 'writer', 'review']

In [0]:
def fill_na(df, features):
    for feature in features:
        df[feature].fillna('', inplace=True)

def preprocess_data_part(data_part):
    data_part = data_part.drop(auxiliary_columns, axis=1)

    fill_na(data_part, cat_features)
    fill_na(data_part, text_features)

    X = data_part.drop(['rating_10'], axis=1)
    y = data_part['rating_10']
    return X, y

X_train, y_train = preprocess_data_part(train_df)
X_test, y_test = preprocess_data_part(test_df)

X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

In [60]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

[['cats', 'so', 'cute'],
 ['mouse', 'scare'],
 ['cat', 'defeated', 'mouse'],
 ['cute', 'mice', 'gather', 'army'],
 ['army', 'mice', 'defeated', 'cat'],
 ['cat', 'offers', 'peace'],
 ['cat', 'scared'],
 ['cat', 'mouse', 'live', 'peace']]

In [0]:
def fill_na(df, features):
    for feature in features:
        df[feature].fillna('', inplace=True)

def preprocess_data_part(data_part):
    data_part = data_part.drop(auxiliary_columns, axis=1)

    fill_na(data_part, cat_features)
    fill_na(data_part, text_features)

    X = data_part.drop(['rating_10'], axis=1)
    y = data_part['rating_10']
    return X, y

X_train, y_train = preprocess_data_part(train_df)
X_test, y_test = preprocess_data_part(test_df)

X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

In [0]:
from catboost import CatBoostClassifier

def fit_model(train_pool, validation_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=3000,
        learning_rate=0.05,
        depth=4,
        eval_metric='Accuracy',
        task_type='GPU',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=100,
    )

In [63]:
train_pool = Pool(
    X_train, y_train, 
    cat_features=cat_features,
    text_features=text_features,
)

validation_pool = Pool(
    X_test, y_test, 
    cat_features=cat_features,
    text_features=text_features,
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool)

Train dataset shape: (32712, 16)

0:	learn: 0.3804109	test: 0.3913681	best: 0.3913681 (0)	total: 98.2ms	remaining: 4m 54s
100:	learn: 0.4376987	test: 0.4495660	best: 0.4501773 (96)	total: 6.61s	remaining: 3m 9s
200:	learn: 0.4478173	test: 0.4571464	best: 0.4572686 (193)	total: 12.2s	remaining: 2m 49s
300:	learn: 0.4547567	test: 0.4608143	best: 0.4621592 (279)	total: 17.5s	remaining: 2m 36s
400:	learn: 0.4607483	test: 0.4617924	best: 0.4626482 (394)	total: 22.8s	remaining: 2m 27s
500:	learn: 0.4658229	test: 0.4627705	best: 0.4637486 (483)	total: 28.2s	remaining: 2m 20s
600:	learn: 0.4700110	test: 0.4647267	best: 0.4650935 (576)	total: 33.3s	remaining: 2m 12s
700:	learn: 0.4734654	test: 0.4654603	best: 0.4680279 (654)	total: 38.4s	remaining: 2m 5s
800:	learn: 0.4764612	test: 0.4674166	best: 0.4680279 (654)	total: 43.5s	remaining: 1m 59s
900:	learn: 0.4792737	test: 0.4666830	best: 0.4680279 (654)	total: 48.6s	remaining: 1m 53s
1000:	learn: 0.4820555	test: 0.4653381	best: 0.4680279 (654)	t

In [0]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['Accuracy']
    second_accuracy = second_model.best_score_['validation']['Accuracy']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))





In [65]:
print_score_diff(model_no_text, model)

0.4565350287321188 vs 0.4691282552879325 (+2.76%)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/events/2020_06_04_catboost_tutorial/text_features.ipynb)

**Set GPU as hardware accelerator**

First of all, you need to select GPU as hardware accelerator. There are two simple steps to do so:
Step 1. Navigate to **Runtime** menu and select **Change runtime type**
Step 2. Choose **GPU** as hardware accelerator.
That's all!

Let's install CatBoost.

In [1]:
!pip install catboost

     |████████████████████████████████| 64.8MB 45kB/s 


In this tutorial we will use dataset **Rotten Tomatoes Movie Reviews** from [Kaggle](https://www.kaggle.com) competition for our experiments. Data can be downloaded [here](https://www.kaggle.com/rpnuser8182/rotten-tomatoes/data).

In [43]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

0.23.2


## Reading the data

In [44]:
from catboost.datasets import rotten_tomatoes

train_df, test_df = rotten_tomatoes()

train_df.head(2)

,id,synopsis,rating_MPAA,genre,director,writer,theater_date,dvd_date,box_office,runtime,studio,dvd_date_int,theater_date_int,review,rating,fresh,critic,top_critic,publisher,date,date_int,rating_10
0,830.0,A gay New Yorker stages a marriage of convenie...,R,Art House and International | Comedy | Drama |...,Ang Lee,Ang Lee | James Schamus | Neil Peng,1993-08-04,2004-06-15,NaN,111.0,NaN,20040615,19930804,NaN,0.800000,fresh,Carol Cling,0,Las Vegas Review-Journal,2004-04-16,20040416.0,8.0
1,1161.0,Screenwriter Nimrod Antal makes an impressive ...,R,Action and Adventure | Art House and Internati...,NaN,NaN,2005-04-01,2005-08-30,116783.0,105.0,ThinkFilm Inc.,20050830,20050401,"One very long, dark ride.",0.647059,rotten,NaN,0,E! Online,2005-04-22,20050422.0,6.0


### Features description 

|Id | Feature name      |  Description                                                                                 |
|---|-------------------|----------------------------------------------------------------------------------------------|
| 1 | ``id``            |  unique movie id                                                                             |
| 2 | ``synopsis``      |  brief summary of the major points of a movie                                                |
| 3 | ``rating_MPAA``   |  film rating by MPAA rating system                                                           |
| 4 | ``genre``         |  list of genres that are suitable for this film (e.g. Action, Adventure, Comedy,...          |
| 5 | ``director``      |  list of persons who direct the making of a film                                             |
| 6 | ``writer``        |  list of persons who write a screenplay                                                      |
| 7 | ``theater_date``  |  the date when film was first shown to the public in cinema (string)                         |
| 8 | ``dvd_date``      |  the date when film was released on DVD (string)                                             |
| 9 | ``box_office``    |  the amount of money raised by ticket sales (revenue)                                        |
| 10 | ``runtime``      |  film duration in minutes                                                                    |
| 11 | ``studio``       |  is a major entertainment company or motion picture company (20th Century Fox, Sony Pictures)|
| 12 | ``dvd_date_int`` |  the date when film was released on DVD (converted to integer)                               |
| 13 | ``theater_date_int`` |  the date when film was first shown to the public in cinema (converted to integer)       |
| 14 | ``review``       |  review of a movie, that was written by a critic                                             |
| 15 | ``rating``       |  float rating from 0 to 1 of the film according to the Rotten tomatoes web site              |
| 16 | ``fresh``        |  freshness of review - fresh or rotten                                                       |
| 17 | ``critic``       |  name of reviewer                                                                            |
| 18 | ``top_critic``   |  binary feature, is reviewer a top critic or not                                             |
| 19 | ``publisher``    |  journal or website where the review was published                                           |
| 20 | ``date``         |  the date when critic publish review (string)                                                |
| 21 | ``date_int``     |  the date when critic publish review (converted to integer)                                  |
| 22 | ``rating_10``    |  integer rating from 0 to 10 of the film according to the critic                             |

We mark as **auxiliary** columnns 'id' and 'rating', because they can be the reason of overfitting, 'theater_date','dvd_date','date' because we convert them into integers.

We mark as **text** features 'synopsis' because it is short *text* description of a film, 'genre' because it is combination of categories (we know that strings have structure where words define categories), for example 'Action | Comedy | Adventure', 'director' and 'writer' features are included to the text features by the same reason, 'review' becuase it is a *text* summary of critic opinion.

We mark as **categorical** features 'rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic' and 'publisher' because they can not be splitted into the group of categorical features and feature values can not be compared.

The other columns considered as **numeric**.

## Preparing the data

In [0]:
auxiliary_columns = ['id', 'theater_date', 'dvd_date', 'rating', 'date']
cat_features = ['rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic', 'publisher']
text_features = ['synopsis', 'genre', 'director', 'writer', 'review']

In [0]:
def fill_na(df, features):
    for feature in features:
        df[feature].fillna('', inplace=True)

def preprocess_data_part(data_part):
    data_part = data_part.drop(auxiliary_columns, axis=1)

    fill_na(data_part, cat_features)
    fill_na(data_part, text_features)

    X = data_part.drop(['rating_10'], axis=1)
    y = data_part['rating_10']
    return X, y

X_train, y_train = preprocess_data_part(train_df)
X_test, y_test = preprocess_data_part(test_df)

X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

In [39]:
X_train_no_text.head(2)

,rating_MPAA,box_office,runtime,studio,dvd_date_int,theater_date_int,fresh,critic,top_critic,publisher,date_int
0,R,NaN,111.0,,20040615,19930804,fresh,Carol Cling,0,Las Vegas Review-Journal,20040416.0
1,R,116783.0,105.0,ThinkFilm Inc.,20050830,20050401,rotten,,0,E! Online,20050422.0


In [49]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

[['cats', 'so', 'cute'],
 ['mouse', 'scare'],
 ['cat', 'defeated', 'mouse'],
 ['cute', 'mice', 'gather', 'army'],
 ['army', 'mice', 'defeated', 'cat'],
 ['cat', 'offers', 'peace'],
 ['cat', 'scared'],
 ['cat', 'mouse', 'live', 'peace']]

In [50]:
from catboost import Pool

train_pool_no_text = Pool(
    X_train_no_text, y_train, 
    cat_features=cat_features, 
)

validation_pool_no_text = Pool(
    X_test_no_text, y_test, 
    cat_features=cat_features, 
)

print('Train dataset shape: {}\n'.format(train_pool_no_text.shape))

Train dataset shape: (32712, 11)



In [51]:
from catboost import CatBoostClassifier

def fit_model(train_pool, validation_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=3000,
        learning_rate=0.05,
        depth=4,
        eval_metric='Accuracy',
        task_type='GPU',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=validation_pool,
        verbose=100,
    )

model_no_text = fit_model(train_pool_no_text, validation_pool_no_text)

0:	learn: 0.3804109	test: 0.3913681	best: 0.3913681 (0)	total: 26.4ms	remaining: 1m 19s
100:	learn: 0.4177366	test: 0.4247463	best: 0.4258467 (99)	total: 2.36s	remaining: 1m 7s
200:	learn: 0.4305148	test: 0.4334271	best: 0.4334271 (200)	total: 4.65s	remaining: 1m 4s
300:	learn: 0.4378210	test: 0.4372173	best: 0.4374618 (290)	total: 6.94s	remaining: 1m 2s
400:	learn: 0.4443935	test: 0.4384399	best: 0.4399071 (345)	total: 9.2s	remaining: 59.7s
500:	learn: 0.4505380	test: 0.4394180	best: 0.4402739 (469)	total: 11.4s	remaining: 57.1s
600:	learn: 0.4551846	test: 0.4434527	best: 0.4440641 (597)	total: 13.7s	remaining: 54.7s
700:	learn: 0.4590670	test: 0.4445531	best: 0.4449199 (628)	total: 15.9s	remaining: 52.1s
800:	learn: 0.4637136	test: 0.4451644	best: 0.4452867 (715)	total: 18.1s	remaining: 49.8s
900:	learn: 0.4673820	test: 0.4460203	best: 0.4480988 (860)	total: 20.4s	remaining: 47.5s
1000:	learn: 0.4729457	test: 0.4485878	best: 0.4488324 (985)	total: 22.7s	remaining: 45.3s
1100:	learn: 

KeyboardInterrupt: ignored

# Text Features

In [10]:
train_pool = Pool(
    X_train, y_train, 
    cat_features=cat_features,
    text_features=text_features,
)

validation_pool = Pool(
    X_test, y_test, 
    cat_features=cat_features,
    text_features=text_features,
)

print('Train dataset shape: {}\n'.format(train_pool.shape))

model = fit_model(train_pool, validation_pool)

Train dataset shape: (32712, 16)

0:	learn: 0.3858217	test: 0.3980927	best: 0.3980927 (0)	total: 184ms	remaining: 3m 3s
100:	learn: 0.4529225	test: 0.4584913	best: 0.4584913 (99)	total: 12.5s	remaining: 1m 51s
200:	learn: 0.4671986	test: 0.4649713	best: 0.4649713 (200)	total: 22.3s	remaining: 1m 28s
300:	learn: 0.4773783	test: 0.4671720	best: 0.4671720 (300)	total: 31.8s	remaining: 1m 13s
400:	learn: 0.4850514	test: 0.4670498	best: 0.4686392 (325)	total: 41.6s	remaining: 1m 2s
500:	learn: 0.4941306	test: 0.4675388	best: 0.4691283 (432)	total: 51.5s	remaining: 51.2s
600:	learn: 0.5018342	test: 0.4680279	best: 0.4694950 (584)	total: 1m	remaining: 40.4s
700:	learn: 0.5083761	test: 0.4691283	best: 0.4701064 (694)	total: 1m 10s	remaining: 29.9s
800:	learn: 0.5156823	test: 0.4686392	best: 0.4702286 (731)	total: 1m 19s	remaining: 19.7s
900:	learn: 0.5233859	test: 0.4694950	best: 0.4702286 (731)	total: 1m 28s	remaining: 9.76s
999:	learn: 0.5297444	test: 0.4691283	best: 0.4702286 (731)	total: 1

In [0]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['Accuracy']
    second_accuracy = second_model.best_score_['validation']['Accuracy']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))

In [12]:
print_score_diff(model_no_text, model)

0.4565350287321188 vs 0.47022863430737255 (+3.00%)


<span style="color:red">Note!</span>

1. Text features also cannot contain NaN values, so we converted them into strings manually.
2. The training may be performed only with classification losses and targets.

## How it works?

1. **Text Tokenization**
2. **Dictionary Creation**
3. **Feature Calculation**

## Text Tokenization

Usually we get our text as a sequence of Unicode symbols. So, if the task isn't a DNA classification we don't need such granularity, moreover, we need to extract more complicated entities, e.g. words. The process of extraction tokens -- words, numbers, punctuation symbols or special symbols which defines emoji from a sequence is called **tokenization**.<br>

Tokenization is the first part of text preprocessing in CatBoost and performed as a simple splitting a sequence on a string pattern (e.g. space).

In [0]:
text_small = [
    "Cats are so cute :)",
    "Mouse scare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is scared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [14]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

simple_tokenized_text = tokenize_texts(text_small)
simple_tokenized_text

[['Cats', 'are', 'so', 'cute', ':)'],
 ['Mouse', 'scare...'],
 ['The', 'cat', 'defeated', 'the', 'mouse'],
 ['Cute:', 'Mice', 'gather', 'an', 'army!'],
 ['Army', 'of', 'mice', 'defeated', 'the', 'cat', ':('],
 ['Cat', 'offers', 'peace'],
 ['Cat', 'is', 'scared', ':('],
 ['Cat', 'and', 'mouse', 'live', 'in', 'peace', ':)']]

### More preprocessing!

Lets take a closer look on the tokenization result of small text example -- the tokens contains a lot of mistakes:

1. They are glued with punctuation 'Cute:', 'army!', 'skare...'.
2. The words 'Cat' and 'cat', 'Mice' and 'mice' seems to have same meaning, perhaps they should be the same tokens.
3. The same problem with tokens 'are'/'is' -- they are inflected forms of same token 'be'.

**Punctuation handling**, **lowercasing**, and **lemmatization** processes help to overcome these problems.

### Punctuation handling and lowercasing

In [15]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number']
)

tokenized_text = [tokenizer.tokenize(text) for text in text_small]
tokenized_text

[['cats', 'are', 'so', 'cute'],
 ['mouse', 'scare'],
 ['the', 'cat', 'defeated', 'the', 'mouse'],
 ['cute', 'mice', 'gather', 'an', 'army'],
 ['army', 'of', 'mice', 'defeated', 'the', 'cat'],
 ['cat', 'offers', 'peace'],
 ['cat', 'is', 'scared'],
 ['cat', 'and', 'mouse', 'live', 'in', 'peace']]

### Removing stop words

**Stop words** - the words that are considered to be uninformative in this task, e.g. function words such as *the, is, at, which, on*.
Usually stop words are removed during text preprocessing to reduce the amount of information that is considered for further algorithms.
Stop words are collected manually (in dictionary form) or automatically, for example taking the most frequent words.

In [16]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

[['cats', 'so', 'cute'],
 ['mouse', 'scare'],
 ['cat', 'defeated', 'mouse'],
 ['cute', 'mice', 'gather', 'army'],
 ['army', 'mice', 'defeated', 'cat'],
 ['cat', 'offers', 'peace'],
 ['cat', 'scared'],
 ['cat', 'mouse', 'live', 'peace']]

### Lemmatization

Lemma (Wikipedia) -- is the canonical form, dictionary form, or citation form of a set of words.<br>
For example, the lemma "go" represents the inflected forms "go", "goes", "going", "went", and "gone".<br>
The process of convertation word to its lemma called **lemmatization**.


In [17]:
import nltk

nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
nltk.data.path.append(nltk_data_path)
nltk.download('wordnet', nltk_data_path)

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_tokens_nltk(tokens):
    return list(map(lambda t: lemmatizer.lemmatize(t), tokens))

[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/lib/python3.6/dist-
[nltk_data]     packages/nltk/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [18]:
text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in tokenized_text_no_stop]
text_small_lemmatized_nltk

[['cat', 'so', 'cute'],
 ['mouse', 'scare'],
 ['cat', 'defeated', 'mouse'],
 ['cute', 'mouse', 'gather', 'army'],
 ['army', 'mouse', 'defeated', 'cat'],
 ['cat', 'offer', 'peace'],
 ['cat', 'scared'],
 ['cat', 'mouse', 'live', 'peace']]

Now words with same meaning represented by the same token, tokens are not glued with punctuation.

<span style="color:red">Be carefull.</span> You should verify for your own task:<br>
Is it realy necessary to remove punctuation, lowercasing sentences or performing a lemmatization and/or by word tokenization?<br>

### Let's check up accuracy with new text preprocessing

Since CatBoost doesn't perform spacing punctuation, lowercasing letters and lemmatization, we need to preprocess text manually and then pass it to learning algorithm.

Since the natural text features is only synopsis and review, we will preprocess only them.

In [40]:
%%time

def preprocess_data(X):
    X_preprocessed = X.copy()
    for feature in ['synopsis', 'review']:
        X_preprocessed[feature] = X[feature].apply(lambda x: ' '.join(lemmatize_tokens_nltk(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

train_processed_pool = Pool(
    X_preprocessed_train, y_train, 
    cat_features=cat_features,
    text_features=text_features,
)

validation_processed_pool = Pool(
    X_preprocessed_test, y_test, 
    cat_features=cat_features,
    text_features=text_features,
)

CPU times: user 27.2 s, sys: 405 µs, total: 27.2 s
Wall time: 27.2 s


In [20]:
model_on_processed_data = fit_model(train_processed_pool, validation_processed_pool)

0:	learn: 0.3858217	test: 0.3980927	best: 0.3980927 (0)	total: 180ms	remaining: 2m 59s
100:	learn: 0.4542064	test: 0.4589803	best: 0.4589803 (100)	total: 12.7s	remaining: 1m 53s
200:	learn: 0.4693079	test: 0.4663162	best: 0.4674166 (192)	total: 22.8s	remaining: 1m 30s
300:	learn: 0.4810773	test: 0.4721849	best: 0.4726739 (289)	total: 32.5s	remaining: 1m 15s
400:	learn: 0.4899120	test: 0.4763419	best: 0.4763419 (400)	total: 42.4s	remaining: 1m 3s
500:	learn: 0.4982881	test: 0.4747524	best: 0.4763419 (400)	total: 51.9s	remaining: 51.7s
600:	learn: 0.5054414	test: 0.4752415	best: 0.4767086 (583)	total: 1m 1s	remaining: 40.8s
700:	learn: 0.5132979	test: 0.4749969	best: 0.4767086 (583)	total: 1m 11s	remaining: 30.3s
800:	learn: 0.5199621	test: 0.4771977	best: 0.4771977 (800)	total: 1m 20s	remaining: 20s
900:	learn: 0.5278185	test: 0.4765864	best: 0.4775645 (801)	total: 1m 30s	remaining: 9.9s
999:	learn: 0.5356444	test: 0.4756083	best: 0.4775645 (801)	total: 1m 39s	remaining: 0us
bestTest = 

In [21]:
print_score_diff(model, model_on_processed_data)

0.47022863430737255 vs 0.47756449443697274 (+1.56%)


## Dictionary Creation

After the first stage, preprocessing of text and tokenization, the second stage starts. The second stage uses the prepared text to select a set of units, which will be used for building new numerical features.

A set of selected units is called dictionary. It might contain words, word bigramms, or character n-gramms.

In [0]:
from catboost.text_processing import Dictionary

In [0]:
dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=10)

dictionary.fit(text_small_lemmatized_nltk);
#dictionary.fit(text_small, tokenizer)

In [0]:
dictionary.save('dictionary.tsv')
!cat dictionary.tsv

## Feature Calculation

### Convertation into fixed size vectors

The majority of classic ML algorithms are computing and performing predictions on a fixed number of features $F$.<br>
That means that learning set $X = \{x_i\}$ contains vectors $x_i = (a_0, a_1, ..., a_F)$ where $F$ is constant.

Since text object $x$ is not a fixed length vector, we need to perform preprocessing of the origin set $D$.<br>
One of the simplest text to vector encoding technique is **Bag of words (BoW)**.

### Bag of words algorithm

The algorithm takes in a dictionary and a text.<br>
During the algorithm text $x = (a_0, a_1, ..., a_k)$ converted into vector $\tilde x = (b_0, b_1, ..., b_F)$,<br> where $b_i$ is 0/1 (depending on whether there is a word with id=$i$ from dictionary into text $x$).

In [24]:
text_small_lemmatized_nltk

[['cat', 'so', 'cute'],
 ['mouse', 'scare'],
 ['cat', 'defeated', 'mouse'],
 ['cute', 'mouse', 'gather', 'army'],
 ['army', 'mouse', 'defeated', 'cat'],
 ['cat', 'offer', 'peace'],
 ['cat', 'scared'],
 ['cat', 'mouse', 'live', 'peace']]

In [25]:
dictionary.apply([text_small_lemmatized_nltk[0]])

[[0, 3]]

In [26]:
def bag_of_words(tokenized_text, dictionary):
    features = np.zeros((len(tokenized_text), dictionary.size))
    for i, tokenized_sentence in enumerate(tokenized_text):
        indices = np.array(dictionary.apply([tokenized_sentence])[0])
        features[i, indices] = 1
    return features

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
bow_features

array([[1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 1., 0., 0., 0.],
       [1., 1., 1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 1., 0., 1., 0., 0.]])

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.metrics import log_loss

def fit_linear_model(X, c):
    model = LogisticRegression()
    model.fit(X, c)
    return model

def fit_naive_bayes(X, c):
    clf = MultinomialNB()
    if isinstance(X, csr_matrix):
        X.eliminate_zeros()
    clf.fit(X, c)
    return clf

def evaluate_model_logloss(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = log_loss(y, y_pred)
    print('Logloss: ' + str(metric))

In [28]:
def evaluate_models(X, y):
    linear_model = fit_linear_model(bow_features, target_small)
    naive_bayes = fit_naive_bayes(bow_features, target_small)
        
    print('Linear model')
    evaluate_model_logloss(linear_model, X, y)
    print('Naive bayes')
    evaluate_model_logloss(naive_bayes, X, y)
    print('Comparing to constant prediction')
    logloss_constant_prediction = log_loss(y, np.ones(shape=(len(text_small), 2)) * 0.5)
    print('Logloss: ' + str(logloss_constant_prediction))
    
evaluate_models(bow_features, target_small)

Linear model
Logloss: 0.5003949004139002
Naive bayes
Logloss: 0.4528488772318392
Comparing to constant prediction
Logloss: 0.6931471805599453


In [29]:
dictionary = Dictionary(occurence_lower_bound=0)
dictionary.fit(text_small_lemmatized_nltk)

bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

Linear model
Logloss: 0.46515559462967115
Naive bayes
Logloss: 0.3680393546716464
Comparing to constant prediction
Logloss: 0.6931471805599453


In [42]:
bow_features.shape

(8, 29)

In [0]:
def fit_linear_model(X, c):
    model = LogisticRegression()
    model.fit(X, c)
    return model

def fit_naive_bayes(X, c):
    clf = MultinomialNB()
    if isinstance(X, csr_matrix):
        X.eliminate_zeros()
    clf.fit(X, c)
    return clf

def evaluate_model_logloss(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = log_loss(y, y_pred)
    print('Logloss: ' + str(metric))

### Looking at sequences of letters / words

Let's look at the example: texts 'The cat defeated the mouse' and 'Army of mice defeated the cat :('<br>
Simplifying it we have three tokens in each sentence 'cat defeat mouse' and 'mouse defeat cat'.<br>
After applying BoW we get two equal vectors with the opposite meaning:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

How to distinguish them?
Lets add sequences of words as a single tokens into our dictionary:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** is a continguous sequence of $n$ items from a given sample of text or speech (Wikipedia).<br>
In example above Bi-gram (Bigram) = 2-gram of words.

Ngrams help to add into vectors more information about text structure, moreover there are n-grams has no meanings in separation, for example, 'Mickey Mouse company'.

In [0]:
dictionary = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary.fit(text_small_lemmatized_nltk)

dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [0]:
bow_features = bag_of_words(text_small_lemmatized_nltk, dictionary)
evaluate_models(bow_features, target_small)

### Unigram + Bigram

In [31]:
dictionary1 = Dictionary(occurence_lower_bound=0)
dictionary1.fit(text_small_lemmatized_nltk)

bow_features1 = bag_of_words(text_small_lemmatized_nltk, dictionary1)

dictionary2 = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary2.fit(text_small_lemmatized_nltk)

bow_features2 = bag_of_words(text_small_lemmatized_nltk, dictionary2)

bow_features = np.concatenate((bow_features1, bow_features2), axis=1)
evaluate_models(bow_features, target_small)

Linear model
Logloss: 0.3226184546657689
Naive bayes
Logloss: 0.13103685350656918
Comparing to constant prediction
Logloss: 0.6931471805599453


## CatBoost Configuration

Parameter names:

1. **Text Tokenization** - `tokenizers`
2. **Dictionary Creation** - `dictionaries`
3. **Feature Calculation** - `feature_calcers`

\* More complex configuration with `text_processing` parameter

### `tokenizers`

Tokenizers used to preprocess Text type feature columns before creating the dictionary.

[Documentation](https://catboost.ai/docs/references/tokenizer_options.html).

```
tokenizers = [{
	'tokenizerId': 'Space',
	'delimiter': ' ',
	'separator_type': 'ByDelimiter',
},{
	'tokenizerId': 'Sense',
	'separator_type': 'BySense',
}]
```

### `dictionaries`

Dictionaries used to preprocess Text type feature columns.

[Documentation](https://catboost.ai/docs/references/dictionaries_options.html).

```
dictionaries = [{
	'dictionaryId': 'Unigram',
	'max_dictionary_size': '50000',
	'gram_count': '1',
},{
	'dictionaryId': 'Bigram',
	'max_dictionary_size': '50000',
	'gram_count': '2',
},{
	'dictionaryId': 'Trigram',
	'token_level_type': 'Letter',
	'max_dictionary_size': '50000',
	'gram_count': '3',
}]
```

### `feature_calcers`

Feature calcers used to calculate new features based on preprocessed Text type feature columns.

1. **`BoW`**<br>
Bag of words: 0/1 features (text sample has or not token_id).<br>
Number of produced numeric features = dictionary size.<br>
Parameters: `top_tokens_count` - maximum number of tokens that will be used for vectorization in bag of words, the most frequent $n$ tokens are taken (**highly affect both on CPU ang GPU RAM usage**).

2. **`NaiveBayes`**<br>
NaiveBayes: [Multinomial naive bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) model. As many new features as classes are added. This feature is calculated by analogy with counters in CatBoost by permutation ([estimation of CTRs](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html)). In other words, a random permutation is made and then we go from top to bottom on the dataset and calculate the probability of its belonging to this class for each object.

3. **`BM25`**<br>
[BM25](https://en.wikipedia.org/wiki/Okapi_BM25). As many new features as classes are added. The idea is the same as in Naive Bayes, but for each class we calculate not the conditional probability, but a certain relevance, which is similar to tf-idf, where the tokens instead of the words and the classes instead of the documents (or rather, the unification of all texts of this class). Only the tf multiplier in BM25 is replaced with another multiplier, which gives an advantage to classes that contain rare tokens.

```
feature_calcers = [
	'BoW:top_tokens_count=1000',
	'NaiveBayes',
	'BM25',
]
```

### `text_processing`

```
text_processing = {
    "tokenizers" : [{
        "tokenizer_id" : "Space",
        "separator_type" : "ByDelimiter",
        "delimiter" : " "
    }],

    "dictionaries" : [{
        "dictionary_id" : "BiGram",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "2"
    }, {
        "dictionary_id" : "Word",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "1"
    }],

    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["BiGram", "Word"],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["Space"]
        }, {
            "dictionaries_names" : ["Word"],
            "feature_calcers" : ["NaiveBayes"],
            "tokenizers_names" : ["Space"]
        }],
    }
}
```

In [32]:
model_on_processed_data_2 = fit_model(
    train_processed_pool,
    validation_processed_pool,
    text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],
    
        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "2"
        }, {
            "dictionary_id" : "Word",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "1"
        }],
    
        "feature_processing" : {
            "default" : [{
                "dictionaries_names" : ["BiGram", "Word"],
                "feature_calcers" : ["BoW"],
                "tokenizers_names" : ["Space"]
            }, {
                "dictionaries_names" : ["Word"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            }],
        }
    }
)

0:	learn: 0.3858217	test: 0.3980927	best: 0.3980927 (0)	total: 180ms	remaining: 2m 59s
100:	learn: 0.4542064	test: 0.4589803	best: 0.4589803 (100)	total: 12.6s	remaining: 1m 52s
200:	learn: 0.4693079	test: 0.4663162	best: 0.4674166 (192)	total: 22.6s	remaining: 1m 29s
300:	learn: 0.4810773	test: 0.4721849	best: 0.4726739 (289)	total: 32.3s	remaining: 1m 15s
400:	learn: 0.4899120	test: 0.4763419	best: 0.4763419 (400)	total: 42.2s	remaining: 1m 3s
500:	learn: 0.4982881	test: 0.4747524	best: 0.4763419 (400)	total: 51.7s	remaining: 51.5s
600:	learn: 0.5054414	test: 0.4752415	best: 0.4767086 (583)	total: 1m 1s	remaining: 40.7s
700:	learn: 0.5132979	test: 0.4749969	best: 0.4767086 (583)	total: 1m 10s	remaining: 30.2s
800:	learn: 0.5199621	test: 0.4771977	best: 0.4771977 (800)	total: 1m 20s	remaining: 20s
900:	learn: 0.5278185	test: 0.4765864	best: 0.4775645 (801)	total: 1m 29s	remaining: 9.85s
999:	learn: 0.5356444	test: 0.4756083	best: 0.4775645 (801)	total: 1m 39s	remaining: 0us
bestTest =

In [33]:
print_score_diff(model_no_text, model_on_processed_data_2)

0.4565350287321188 vs 0.47756449443697274 (+4.61%)


# Summary: Text features in CatBoost

### The algorithm:
1. Input text is loaded as a usual column. ``text_column: [string]``.
2. Each text sample is tokenized via splitting by space. ``tokenized_column: [[string]]``.
3. Dictionary estimation.
4. Each string in tokenized column is converted into token_id from dictionary. ``text: [[token_id]]``.
5. Depending on the parameters CatBoost produce features basing on the resulting text column: Bag of words, Multinomial naive bayes or Bm25.
6. Computed float features are passed into the usual CatBoost learning algorithm.